In [1]:
# python -m pip install prophet
# here i will try prophet on the loged data so log(x)

In [110]:
import numpy as np
import pandas as pd
import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
import plotly.express as px
from datetime import datetime,date
from datetime import timedelta

%matplotlib inline
init_notebook_mode(connected=True)
cf.set_config_file(sharing='private',theme='pearl',offline=True)
#print("Cufflinks Version : {}".format(cf.__version__))

# facebook prophet requires the data to be in 2 colums named ds and y
# ds will be a datetime column with no timezone data and y the label we are forecasting
# ds must NOT be an index like other machine learning algorithms
df = pd.read_csv('./data/btc-usd-max latest.csv', index_col=0, parse_dates=True )
df = df.resample('D').first()
df.drop(['market_cap', 'total_volume'], axis=1, inplace=True)
df['price'].fillna(method='ffill', inplace=True)

import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')


dynamic_days = 1550

In [149]:
def train_test_split(data, percentage):
    """
    Create train test datasets based on the percentage above to split
    """
    
    data2 = data.values
    n = int(len(data2) * (1-percentage))
    return data2[:n], data2[n:] , n

In [112]:
df['target'] = df.price.shift(-1)
df.dropna(inplace=True)
df.head()

,price,target
snapped_at,,
2013-04-28 00:00:00+00:00,135.30,141.96
2013-04-29 00:00:00+00:00,141.96,135.30
2013-04-30 00:00:00+00:00,135.30,117.00
2013-05-01 00:00:00+00:00,117.00,103.43
2013-05-02 00:00:00+00:00,103.43,91.01


In [113]:
df['price'].iplot();

In [150]:
train, test, idx = train_test_split(df, 0.2)

In [115]:
X = train[:,:-1]
y = train[:,-1]

In [120]:
from xgboost import XGBRegressor

model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
model.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [123]:
# create a predict method to predict one sample at a time
def xgb_predict(train,val):
    train = np.array(train)
    X,y =  train[:,:-1], train[:,-1]
    model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
    model.fit(X,y)
    val = np.array(val). reshape(1,-1)
    pred = model.predict(val)
    return pred[0]

In [125]:
# validate data forward with RMSE

from sklearn.metrics import mean_squared_error

def validate_model(data, perc):
    predictions = []
    history = [x for x in train]
    
    for i in range(len(test)):
        test_x , test_y = test[i,:-1], test[i,-1]
        pred = xgb_predict(history, test_x[0])
        predictions.append(pred)
        history.append(test[i])
    error = mean_squared_error(test[:,-1], predictions, squared=False)
    return error, test[:,-1], predictions

In [126]:
%%time
rmse, y, pred = validate_model(df, 0.2)

CPU times: total: 2h 55min 42s
Wall time: 27min 13s


In [127]:
rmse

2302.675274543113

In [141]:
test

array([[19091.52418173, 18753.28892168],
       [18753.28892168, 17138.0295124 ],
       [17138.0295124 , 17140.27555252],
       ...,
       [19072.78051336, 19272.92184777],
       [19272.92184777, 19557.89554887],
       [19557.89554887, 19348.02936881]])

In [184]:
df.index[idx] , df['target'].values[idx]

(Timestamp('2020-11-25 00:00:00+0000', tz='UTC', freq='D'), 18753.288921684914)

In [183]:
df

,price,target
snapped_at,,
2013-04-28 00:00:00+00:00,135.300000,141.960000
2013-04-29 00:00:00+00:00,141.960000,135.300000
2013-04-30 00:00:00+00:00,135.300000,117.000000
2013-05-01 00:00:00+00:00,117.000000,103.430000
2013-05-02 00:00:00+00:00,103.430000,91.010000
...,...,...
2022-10-14 00:00:00+00:00,19383.943220,19198.107145
2022-10-15 00:00:00+00:00,19198.107145,19072.780513
2022-10-16 00:00:00+00:00,19072.780513,19272.921848


In [187]:
X

array([[  135.3       ],
       [  141.96      ],
       [  135.3       ],
       ...,
       [18689.52865583],
       [18390.61184379],
       [18360.69330039]])